In [2]:
import numpy as np
import pandas as pd
from scipy.special import gammaln, gamma
from collections import Counter
from scipy.optimize import basinhopping
np.set_printoptions(suppress=True)

In [4]:
# Load the data and process it.

df = pd.read_csv("../mturk_surveys.csv")
df["hit_answered_date"]=pd.to_datetime(df["hit_answered_date"])
df['date'] = pd.to_datetime(df['hit_answered_date'].apply(lambda x : x.date()))

# Calculate the time period of each survey answer. 
# Below we take out time period to be 30-day periods. 
# We can change this to be weeks, months, or anything else
window_length = 30
minDate = min(df['date'])
df['period'] = (df['date'] - minDate).dt.days // window_length 

#df = df[df['period'] < 10]
# Create a list of all unique worker IDs 
workers = pd.DataFrame(list(set(df.worker_id)), columns=['worker_id']).reset_index().set_index('worker_id')

# Fix the issue of duplicate records within the same period.
data = df[['worker_id', 'period']].drop_duplicates().pivot_table(
    index = 'worker_id',
    values = 'period',                    
    aggfunc=('count','min','max')
).join(workers).set_index('index').sort_index()

# Aggregate the capture histories
#data = data.groupby(['min', 'max', 'count']).size().reset_index(name='count_history')
S = df['period'].max() + 1
dat = np.zeros(S + 1)
out = Counter(data['count'].values)
for k,v in out.items():
    dat[k] = v
V = len(workers)
print(S, V)

36 46145


In [11]:
df.drop_duplicates().groupby(['period']).agg(['count'])

,Unnamed: 0,gender,hit_answered_date,hit_creation_date,household_income,household_size,location_city,location_country,location_region,marital_status,post_to_completion_secs,worker_id,year_of_birth,date
,count,count,count,count,count,count,count,count,count,count,count,count,count,count
period,,,,,,,,,,,,,,
0,2825,2825,2825,2808,2825,2825,2825,2825,2825,2825,2808,2825,2825,2825
1,2923,2923,2923,2870,2923,2923,2923,2923,2923,2923,2870,2923,2923,2923
2,2877,2877,2877,2824,2877,2877,2877,2877,2877,2877,2824,2877,2877,2877
3,2742,2742,2742,2675,2742,2742,2742,2742,2742,2742,2675,2742,2742,2742
4,2914,2914,2914,2853,2914,2914,2914,2914,2914,2914,2853,2914,2914,2914
5,2965,2965,2965,2876,2965,2965,2965,2965,2965,2965,2876,2965,2965,2965
6,2947,2947,2947,2871,2947,2947,2947,2947,2947,2947,2871,2947,2947,2947
7,2932,2932,2932,2842,2932,2932,2932,2932,2932,2932,2842,2932,2932,2932
